## 1. 为什么要降维？
为了解决特征矩阵过大，导致计算量过大出现维度灾难，训练时间长等问题，需要降低维度来简化机器学习模型的训练和预测
## 2.常用的降维方法
### 2.1 PCA
PCA(主成分分析)，通过正交变换将一组存在相关性的变量转换为一组线性不相关的变量，即把多指标转化为少数几个综合指标，转换后的这组变量就叫做主成分；其中每个主成分都能够反映原是变量的大部分信息，且所含信息互不重复。

![](img/pca-1.png)

PCA具有如下性质：
1. 保留方差是最大的
2. 最终的重构误差（从变换后回到原始情况）是最小的

下图的例子，需要将2D降为1D，选择不同的投影线得到的结果不同，第1个投影以后方差最大，第3个方差最小，方差越大，能保留的信息更多。
![](img/pca-2.png)

#### 2.1.1 步骤
设有 $m$ 条 $n$ 维的数据
1. 将原始数据按列组成 $m$ 行 $n$ 列矩阵 $X$；
2. 将 $X$ 的每一行(代表一维特征)进行零均值化，即减去这一行的均值；
3. 求出协方差矩阵 $C = \frac{XX^T}{m-1}$；
4. 求出协方差矩阵的特征值和对应的特征向量；
5. 将特征向量按照对应的特征值大小从大到小按行排列成矩阵，取前 $k$ 行组成矩阵 $P$；
6. $Y=PX$ 即为降维到 $k$ 维后的数据。

#### 2.1.2 API
sklearn.decomposition.PCA(n_components=None, *, copy=True, whiten=False, svd_solver='auto', tol=0.0, iterated_power='auto', random_state=None)

**常用参数：**
- n_components: int, float, None 或 string，PCA算法中所要保留的主成分个数，也即保留下来的特征个数，如果 n_components = 1，将把原始数据降到一维；如果赋值为string，如n_components='mle'，将自动选取特征个数，使得满足所要求的方差百分比；如果没有赋值，默认为None，特征个数不会改变（特征数据本身会改变）。
- copy：True 或False，默认为True，即是否需要将原始训练数据复制。
- whiten：True 或False，默认为False，即是否白化，使得每个特征具有相同的方差。
- svd_solver：即指定奇异值分解SVD的方法，由于特征分解是奇异值分解SVD的一个特例，一般的PCA库都是基于SVD实现的。有4个可以选择的值：{‘auto’, ‘full’, ‘arpack’, ‘randomized’}。randomized一般适用于数据量大，数据维度多同时主成分数目比例又较低的PCA降维，它使用了一些加快SVD的随机算法。 full则是传统意义上的SVD，使用了scipy库对应的实现。arpack和randomized的适用场景类似，区别是randomized使用的是scikit-learn自己的SVD实现，而arpack直接使用了scipy库的sparse SVD实现。默认是auto，即PCA类会自己去在前面讲到的三种算法里面去权衡，选择一个合适的SVD算法来降维。一般来说，使用默认值就够了。

#### 2.1.3 属性和方法
**属性：**
- explained_variance_ratio_：返回所保留各个特征的方差百分比，如果n_components没有赋值，则所有特征都会返回一个数值且解释方差之和等于1。
- n_components_：返回所保留的特征个数。

**方法：**
- fit(X): 用数据X来训练PCA模型。
- fit_transform(X)：用X来训练PCA模型，同时返回降维后的数据。
- inverse_transform(newData) ：将降维后的数据转换成原始数据，但可能不会完全一样，会有些许差别。
- transform(X)：将数据X转换成降维后的数据，当模型训练好后，对于新输入的数据，也可以用transform方法来降维。

In [15]:
import numpy as np
from sklearn.decomposition import PCA
X = np.array([[-1, -1, 2], [-2, -1, 4], [-3, -2, 3]])
pca = PCA(n_components=3)
newX = pca.fit_transform(X)
X, newX

(array([[-1, -1,  2],
        [-2, -1,  4],
        [-3, -2,  3]]),
 array([[ 1.44376551e+00, -1.63255223e-01,  7.08267421e-17],
        [-5.11452038e-01,  9.21698391e-01,  7.08267421e-17],
        [-9.32313470e-01, -7.58443167e-01,  7.08267421e-17]]))

In [16]:
pca.explained_variance_ratio_

array([6.88982237e-01, 3.11017763e-01, 3.22484619e-33])

In [17]:
# 可以看出第一个特征99.24%表达整个数据集，因此我们可以降到1维：
pca = PCA(n_components=1)
newX = pca.fit_transform(X)
newX

array([[ 1.44376551],
       [-0.51145204],
       [-0.93231347]])

### 2.2 SVD 奇异值分解
SVD 是对矩阵进行分解，SVD并不要求要分解的矩阵为方阵。假设我们的矩阵A是一个m×n的矩阵，那么我们定义矩阵 $A$ 的SVD为：
$$ A=U \Sigma V^T$$
其中 $U$ 和是一个 $m \times m$ 的方阵， $\Sigma$ 是一个 $m \times n$ 的矩阵，除了主对角线上的元素以外全为 0，主对角线上的每个元素都称为奇异值， $V$ 是一个 $n \times n$ 的方阵。 $U$ 和 $V$ 都是酉矩阵，即满足
$$U^TU=I=V^TV$$
![](img/svd.jpg)



#### 2.2.1 API
sklearn.decomposition.TruncatedSVD(n_components=2, *, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)
通过截断奇异值分解(SVD)进行线性维度降低。与PCA不同的是，在计算奇异值分解之前不对数据进行居中处理。这意味着它可以有效地处理稀疏矩阵。

支持两种算法：一种是快速随机化SVD求解器，另一种是使用 ARPACK 作为求解器

特别是，截断的SVD可以工作在由sklearn.feature_extraction.text中的向量器返回的term count/tf-idf矩阵上。在这种情况下，它被称为潜在语义分析（LSA）。

**常用参数：**
- n_components：int, default = 2，目标输出维度，必须少于特征维数，默认数据是为了可视化
- algorithm：string, default = “randomized”，SVD求解用，arpack或者randomized
- n_iter：int，randomized SVD solver的迭代次数，ARPACK不可用
- random_state：default = None，随机数发生器种子
- tol：float，ARPACK的公差

#### 2.2.2 属性和方法
**属性：**
- components_：返回所保留的特征
- explained_variance_：公差
- explained_variance_ratio_：返回所保留各个特征的方差百分比
- singular_values_：每个选定components的奇异值

**方法：**
- fit(self, X[, y])：通过X拟合LSI
- fit_transform(self, X[, y])：用LSI模型拟合X，并对X进行降维
- inverse_transform(self, X)：将X变换回原来的空间
- transform(self, X)：降维

In [26]:
from sklearn.decomposition import TruncatedSVD
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
svd = TruncatedSVD(n_components=1)
newX = svd.fit_transform(X)
X, newX

(array([[-1, -1],
        [-2, -1],
        [-3, -2],
        [ 1,  1],
        [ 2,  1],
        [ 3,  2]]),
 array([[ 1.38340578],
        [ 2.22189802],
        [ 3.6053038 ],
        [-1.38340578],
        [-2.22189802],
        [-3.6053038 ]]))

In [27]:
svd.explained_variance_

array([6.61628593])

In [28]:
svd.explained_variance_ratio_

array([0.99244289])

In [29]:
svd.components_

array([[-0.83849224, -0.54491354]])